# Large Transactions in the Mempool
Author: Evan Azevedo  
Organization: Amberdata  

## Table of Contents
* Finding Whales on Ethereum from the past year
* Exploring their mempools
* Interpreting the results

In [1]:
# load the packages
import os
import requests
import warnings
import datetime as dt
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv

In [2]:
# helper functions
def get_key():
    "Get the API key from an .env file"
    if ".env" not in os.listdir("../"):
        print("Configuring API Key...")
        key = input("Amberdata API Key: ")
        with open("../.env", "w") as f:
            f.write(f"AMBERDATA_API_KEY={key}\n")
    load_dotenv(verbose=True)
    return {
        "AMBERDATA_API_KEY": os.getenv("AMBERDATA_API_KEY")
    }

def get_response(url, headers=None, queryString=None):
    "Get the REST response from the specified URL"
    if not headers:
        headers = {'x-api-key': api_key["AMBERDATA_API_KEY"]}
    if queryString:
        response = requests.request("GET", url, headers=headers, params=queryString)
    else:
        response = requests.request("GET", url, headers=headers)
    response = json.loads(response.text)
    try:
        if response["title"] == "OK":
            return response["payload"]
    except Exception:
        return None
    
api_key = get_key()

## Whale Hunting

In [13]:
url = "https://web3api.io/api/v2/transactions/"

querystring = {"status":"pending","size":"100","includeFunctions":"false",
               "includeLogs":"true","includeTokenTransfers":"false"}

headers = {
    'x-amberdata-blockchain-id': "ethereum-mainnet",
    'x-api-key': api_key["AMBERDATA_API_KEY"]
    }

response = get_response(url, headers, querystring)

df = pd.DataFrame(response["records"])

In [15]:
df.columns

Index(['blockNumber', 'confirmedAt', 'confirmedHash', 'createdAt', 'from',
       'gasLimit', 'gasPrice', 'hash', 'index', 'input', 'nonce',
       'occurrences', 'publicKey', 'r', 'raw', 's', 'state', 'to', 'updatedAt',
       'v', 'value', 'statusResult', 'timestamp'],
      dtype='object')